<a href="https://colab.research.google.com/github/caarotnee/NER_PhoBert_KLTN/blob/main/NER_PhoBert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Cài đặt lại đúng phiên bản PyTorch và CUDA phù hợp trên Colab
!pip uninstall -y torch torchvision torchaudio
!pip uninstall -y nvidia-cublas-cu12 nvidia-cudnn-cu12 nvidia-cuda-runtime-cu12 nvidia-cuda-cupti-cu12 nvidia-cuda-nvrtc-cu12 nvidia-cufft-cu12 nvidia-curand-cu12 nvidia-cusolver-cu12 nvidia-cusparse-cu12 nvidia-nvjitlink-cu12

# Cài đúng bản torch + CUDA 12.4
!pip install --pre torch torchvision torchaudio --index-url https://download.pytorch.org/whl/nightly/cu124

# Cài đúng các thư viện NVIDIA CUDA phụ trợ yêu cầu bởi torch 2.6.0+cu124
!pip install \
  nvidia-cublas-cu12==12.4.5.8 \
  nvidia-cudnn-cu12==9.1.0.70 \
  nvidia-cuda-runtime-cu12==12.4.127 \
  nvidia-cuda-cupti-cu12==12.4.127 \
  nvidia-cuda-nvrtc-cu12==12.4.127 \
  nvidia-cufft-cu12==11.2.1.3 \
  nvidia-curand-cu12==10.3.5.147 \
  nvidia-cusolver-cu12==11.6.1.9 \
  nvidia-cusparse-cu12==12.3.1.170 \
  nvidia-nvjitlink-cu12==12.4.127

# Cài đặt datasets
!pip install -qq datasets

# Kiểm tra phiên bản PyTorch
!pip show torch -qq


Found existing installation: torch 2.6.0+cu124
Uninstalling torch-2.6.0+cu124:
  Successfully uninstalled torch-2.6.0+cu124
Found existing installation: torchvision 0.21.0+cu124
Uninstalling torchvision-0.21.0+cu124:
  Successfully uninstalled torchvision-0.21.0+cu124
Found existing installation: torchaudio 2.6.0+cu124
Uninstalling torchaudio-2.6.0+cu124:
  Successfully uninstalled torchaudio-2.6.0+cu124
Found existing installation: nvidia-cublas-cu12 12.5.3.2
Uninstalling nvidia-cublas-cu12-12.5.3.2:
  Successfully uninstalled nvidia-cublas-cu12-12.5.3.2
Found existing installation: nvidia-cudnn-cu12 9.3.0.75
Uninstalling nvidia-cudnn-cu12-9.3.0.75:
  Successfully uninstalled nvidia-cudnn-cu12-9.3.0.75
Found existing installation: nvidia-cuda-runtime-cu12 12.5.82
Uninstalling nvidia-cuda-runtime-cu12-12.5.82:
  Successfully uninstalled nvidia-cuda-runtime-cu12-12.5.82
Found existing installation: nvidia-cuda-cupti-cu12 12.5.82
Uninstalling nvidia-cuda-cupti-cu12-12.5.82:
  Successfull

In [2]:
# Tải các file dữ liệu
!gdown 1DuwYRftQjQmQcAR4FVPH-HvGuxGi4ist
!gdown 11xZZfla8CDH54-EeUUdnAAoT2ummuEJh
!gdown 1wVyhQkhAzwod2at7Ir3tRHbdCMOszzwg


Downloading...
From: https://drive.google.com/uc?id=1DuwYRftQjQmQcAR4FVPH-HvGuxGi4ist
To: /content/train_word.conll
100% 1.42M/1.42M [00:00<00:00, 157MB/s]
Downloading...
From: https://drive.google.com/uc?id=11xZZfla8CDH54-EeUUdnAAoT2ummuEJh
To: /content/test_word.conll
100% 958k/958k [00:00<00:00, 138MB/s]
Downloading...
From: https://drive.google.com/uc?id=1wVyhQkhAzwod2at7Ir3tRHbdCMOszzwg
To: /content/dev_word.conll
100% 628k/628k [00:00<00:00, 84.0MB/s]


In [19]:
# để đọc file định dạng CoNLL
def read_conll(file_path):
    sentences = []
    sentence_labels = []
    unique_labels = set()  # To collect unique labels

    with open(file_path, 'r') as file:
        current_sentence_tokens = []
        current_sentence_labels = []

        for line in file:
            line = line.strip()  # Remove leading/trailing whitespace, including '\n'

            # If it's an empty line, sentence boundary detected
            if not line:
                if current_sentence_tokens:  # Check if there's a sentence to append
                    sentences.append(' '.join(current_sentence_tokens))
                    sentence_labels.append(' '.join(current_sentence_labels))
                current_sentence_tokens = []  # Reset for the next sentence
                current_sentence_labels = []  # Reset for the next sentence
            else:
                line_parts = line.split()  # Split line into token and label
                current_sentence_tokens.append(line_parts[0])

                if len(line_parts) >= 2:
                    current_sentence_labels.append(line_parts[1])
                    unique_labels.add(line_parts[1])  # Add label to the set of unique labels
                else:
                    current_sentence_labels.append('O')  # Default to 'O' if no label provided

    # Append the last sentence if the file doesn't end with an empty line
    if current_sentence_tokens:
        sentences.append(' '.join(current_sentence_tokens))
        sentence_labels.append(' '.join(current_sentence_labels))

    print(f"Unique labels found: {unique_labels}")
    return sentences, sentence_labels

# Load the datasets
test_sentences, test_labels = read_conll('./test_word.conll')
dev_sentences, dev_labels = read_conll('./dev_word.conll')
train_sentences, train_labels = read_conll('./train_word.conll')

# Now, test_sentences, test_labels, dev_sentences, dev_labels, train_sentences, and train_labels are arrays of strings


Unique labels found: {'I-ORGANIZATION', 'B-GENDER', 'B-SYMPTOM_AND_DISEASE', 'B-AGE', 'B-PATIENT_ID', 'I-JOB', 'B-DATE', 'I-SYMPTOM_AND_DISEASE', 'O', 'B-ORGANIZATION', 'B-LOCATION', 'I-NAME', 'I-DATE', 'B-JOB', 'I-PATIENT_ID', 'I-LOCATION', 'B-NAME', 'B-TRANSPORTATION', 'I-AGE', 'I-TRANSPORTATION'}
Unique labels found: {'I-ORGANIZATION', 'B-GENDER', 'B-SYMPTOM_AND_DISEASE', 'B-AGE', 'B-PATIENT_ID', 'I-JOB', 'B-DATE', 'I-SYMPTOM_AND_DISEASE', 'O', 'B-ORGANIZATION', 'B-LOCATION', 'I-NAME', 'I-DATE', 'B-JOB', 'I-PATIENT_ID', 'I-LOCATION', 'B-NAME', 'B-TRANSPORTATION', 'I-TRANSPORTATION'}
Unique labels found: {'I-ORGANIZATION', 'B-GENDER', 'B-SYMPTOM_AND_DISEASE', 'B-AGE', 'B-PATIENT_ID', 'I-JOB', 'B-DATE', 'I-SYMPTOM_AND_DISEASE', 'O', 'B-ORGANIZATION', 'B-LOCATION', 'I-NAME', 'I-DATE', 'B-JOB', 'I-PATIENT_ID', 'I-LOCATION', 'B-NAME', 'B-TRANSPORTATION', 'I-AGE', 'I-TRANSPORTATION'}


In [20]:
test_sentences[1]

'Bác_sĩ Trần_Thanh_Linh , từ Bệnh_viện Chợ_Rẫy chi_viện phụ_trách đơn_nguyên hồi_sức tích_cực , cho biết " bệnh_nhân 416 " vẫn đang duy_trì ECMO , thở máy , hiện xơ phổi rất nhiều .'

In [21]:
test_labels[1]

'O O O O B-ORGANIZATION I-ORGANIZATION O O O O O O O O O O B-PATIENT_ID O O O O O O O O O O B-SYMPTOM_AND_DISEASE I-SYMPTOM_AND_DISEASE I-SYMPTOM_AND_DISEASE I-SYMPTOM_AND_DISEASE O'

In [22]:
from datasets import Dataset

# Chuẩn bị dữ liệu từ sentences và labels
def prepare_dataset(sentences, labels):
    return {'tokens': sentences, 'labels': labels}

train_dataset = prepare_dataset(train_sentences, train_labels)
dev_dataset = prepare_dataset(dev_sentences, dev_labels)
test_dataset = prepare_dataset(test_sentences, test_labels)

# Chuyển đổi dữ liệu string thành mảng
def process_string_to_array(dataset):
    return {
        'tokens': [sentence.split() for sentence in dataset['tokens']],
        'labels': [label_seq.split() for label_seq in dataset['labels']]
    }

# Xử lý dữ liệu
train_dataset = process_string_to_array(train_dataset)
dev_dataset = process_string_to_array(dev_dataset)
test_dataset = process_string_to_array(test_dataset)

# Chuyển đổi thành Hugging Face Dataset
train_dataset = Dataset.from_dict(train_dataset)
dev_dataset = Dataset.from_dict(dev_dataset)
test_dataset = Dataset.from_dict(test_dataset)

# In kích thước của các dataset và mẫu để kiểm tra
print(f"Train dataset size: {len(train_dataset)}")
print(f"Dev dataset size: {len(dev_dataset)}")
print(f"Test dataset size: {len(test_dataset)}")
print("Train dataset sample:", train_dataset[0])
print("Dev dataset sample:", dev_dataset[0])
print("Test dataset sample:", test_dataset[0])


Train dataset size: 5027
Dev dataset size: 2000
Test dataset size: 3000
Train dataset sample: {'tokens': ['Đồng_thời', ',', 'bệnh_viện', 'tiếp_tục', 'thực_hiện', 'các', 'biện_pháp', 'phòng_chống', 'dịch_bệnh', 'COVID', '-', '19', 'theo', 'hướng_dẫn', 'của', 'Bộ', 'Y_tế', '.'], 'labels': ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-ORGANIZATION', 'I-ORGANIZATION', 'O']}
Dev dataset sample: {'tokens': ['Bác_sĩ', 'Nguyễn_Trung_Nguyên', ',', 'Giám_đốc', 'Trung_tâm', 'Chống', 'độc', ',', 'Bệnh_viện', 'Bạch_Mai', ',', 'cho', 'biết', 'bệnh_nhân', 'được', 'chuyển', 'đến', 'bệnh_viện', 'ngày', '7/3', ',', 'chẩn_đoán', 'ngộ_độc', 'thuốc', 'điều_trị', 'sốt_rét', 'chloroquine', '.'], 'labels': ['O', 'O', 'O', 'O', 'B-ORGANIZATION', 'I-ORGANIZATION', 'I-ORGANIZATION', 'I-ORGANIZATION', 'I-ORGANIZATION', 'I-ORGANIZATION', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-DATE', 'O', 'O', 'B-SYMPTOM_AND_DISEASE', 'I-SYMPTOM_AND_DISEASE', 'O', 'O', 'O', 'O']}
Test datas

In [23]:

# Step 5: Define an Example class
class Example:
    def __init__(self, words, slot_labels, guid=None):
        self.words = words
        self.slot_labels = slot_labels
        self.guid = guid

# Step 6: Convert the dataset to Example objects
def convert_to_examples(dataset):
    return [
        Example(words=tokens, slot_labels=labels, guid=i)
        for i, (tokens, labels) in enumerate(zip(dataset['tokens'], dataset['labels']))
    ]

# Convert datasets into Example objects
train_examples = convert_to_examples(train_dataset)
dev_examples = convert_to_examples(dev_dataset)
test_examples = convert_to_examples(test_dataset)


In [24]:
import logging
logger = logging.getLogger(__name__)

import copy
import json
import logging
import os

In [25]:

# Định nghĩa chuyển đổi từ examples thành features
def convert_examples_to_features(
    examples,
    max_seq_len,
    tokenizer,
    pad_label_id=-100,
    cls_token_segment_id=0,
    pad_token_segment_id=0,
    sequence_segment_id=0,
    mask_padding_with_zero=True,
):
    # Lấy các token đặc biệt từ tokenizer
    cls_token = tokenizer.cls_token
    sep_token = tokenizer.sep_token
    unk_token = tokenizer.unk_token
    pad_token_id = tokenizer.pad_token_id

    features = []

    for example_index, example in enumerate(examples):
        if example_index % 400 == 0:
            logger.info(f"Processing example {example_index} of {len(examples)}")

        tokens = []
        label_ids = []

        for word, label in zip(example.words, example.slot_labels):
            word_tokens = tokenizer.tokenize(word)

            if not word_tokens:
                word_tokens = [unk_token]

            tokens.extend(word_tokens)
            label_id = label_map[label]
            label_ids.extend([label_id] + [pad_label_id] * (len(word_tokens) - 1))

        special_tokens_count = 2
        if len(tokens) > max_seq_len - special_tokens_count:
            tokens = tokens[:max_seq_len - special_tokens_count]
            label_ids = label_ids[:max_seq_len - special_tokens_count]

        tokens.append(sep_token)
        label_ids.append(pad_label_id)
        token_type_ids = [sequence_segment_id] * len(tokens)

        tokens = [cls_token] + tokens
        label_ids = [pad_label_id] + label_ids
        token_type_ids = [cls_token_segment_id] + token_type_ids

        input_ids = tokenizer.convert_tokens_to_ids(tokens)

        attention_mask = [1 if mask_padding_with_zero else 0] * len(input_ids)

        padding_length = max_seq_len - len(input_ids)
        input_ids += [pad_token_id] * padding_length
        attention_mask += [0 if mask_padding_with_zero else 1] * padding_length
        token_type_ids += [pad_token_segment_id] * padding_length
        label_ids += [pad_label_id] * padding_length

        features.append(
            InputFeatures(
                input_ids=input_ids,
                attention_mask=attention_mask,
                token_type_ids=None,  # Không cần token_type_ids với PhoBERT
                slot_labels_ids=label_ids,
            )
        )

    return features

In [26]:
# Define the label list (ensure that it includes all labels from your dataset)
label_list = ['B-ORGANIZATION', 'B-TRANSPORTATION', 'B-JOB', 'I-PATIENT_ID', 'B-NAME', 'I-DATE', 'O', 'B-PATIENT_ID', 'I-AGE', 'I-JOB', 'B-DATE', 'I-TRANSPORTATION', 'B-SYMPTOM_AND_DISEASE', 'I-SYMPTOM_AND_DISEASE', 'B-GENDER', 'I-NAME', 'B-LOCATION', 'I-LOCATION', 'I-ORGANIZATION', 'B-AGE']

# Create a mapping from label strings to integers
label_map = {label: i for i, label in enumerate(label_list)}


In [27]:
import json

In [28]:
class InputFeatures(object):
    """A single set of features of data."""

    def __init__(self, input_ids, attention_mask, token_type_ids, slot_labels_ids):
        self.input_ids = input_ids
        self.attention_mask = attention_mask
        self.token_type_ids = token_type_ids
        self.slot_labels_ids = slot_labels_ids

    def __repr__(self):
        return str(self.to_json_string())

    def to_dict(self):
        """Serializes this instance to a Python dictionary."""
        output = copy.deepcopy(self.__dict__)
        return output

    def to_json_string(self):
        """Serializes this instance to a JSON string."""
        return json.dumps(self.to_dict(), indent=2, sort_keys=True) + "\n"

In [29]:
!pip install -qq transformers
from transformers import AutoTokenizer

# Dùng tokenizer của PhoBERT
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base", use_fast=False)

# Kiểm tra các token đặc biệt và ID của token pad
tokenizer.cls_token, tokenizer.sep_token, tokenizer.unk_token, tokenizer.pad_token_id


('<s>', '</s>', '<unk>', 1)

In [30]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base")

print("CLS:", tokenizer.cls_token)
print("SEP:", tokenizer.sep_token)
print("UNK:", tokenizer.unk_token)
print("PAD:", tokenizer.pad_token_id)


CLS: <s>
SEP: </s>
UNK: <unk>
PAD: 1


In [31]:
from transformers import AutoTokenizer  # hoặc RobertaTokenizerFast cũng được

# Khởi tạo tokenizer của PhoBERT
tokenizer = AutoTokenizer.from_pretrained('vinai/phobert-base', use_fast=True)

# Đảm bảo thêm khoảng trắng đầu mỗi từ
tokenizer.add_prefix_space = True

# Thiết lập độ dài tối đa
max_seq_len = 128

# Convert examples sang features
train_features = convert_examples_to_features(train_examples, max_seq_len, tokenizer)
dev_features = convert_examples_to_features(dev_examples, max_seq_len, tokenizer)
test_features = convert_examples_to_features(test_examples, max_seq_len, tokenizer)


In [33]:
import torch
from torch.utils.data import Dataset

# Định nghĩa lớp Dataset cho PhoBERT
class NERDataset(Dataset):
    def __init__(self, features):
        self.features = features

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        feature = self.features[idx]
        return {
            'input_ids': torch.tensor(feature.input_ids, dtype=torch.long),
            'attention_mask': torch.tensor(feature.attention_mask, dtype=torch.long),
            # PhoBERT không có token_type_ids
            'labels': torch.tensor(feature.slot_labels_ids, dtype=torch.long),
        }

# Chuyển features thành dataset cho huấn luyện và đánh giá
train_dataset = NERDataset(train_features)
dev_dataset = NERDataset(dev_features)
test_dataset = NERDataset(test_features)


In [34]:
train_dataset[0]

{'input_ids': tensor([   0, 1248,    4,  757,  194,  112,    9,  717, 2137, 3795, 9089, 6232,
         1927,   31, 1195,   63, 1010,    7,  125, 1059,    5,    2,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1]),
 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0,
         0, 0, 0, 0

In [35]:
from transformers import AutoModelForTokenClassification

# Xác định số lượng nhãn (đảm bảo rằng số nhãn của bạn phù hợp với dataset)
num_labels = len(label_list)  # Ví dụ: ['O', 'B-ORG', 'I-ORG', ...]

# Tải mô hình PhoBERT cho tác vụ phân loại token
model = AutoModelForTokenClassification.from_pretrained('vinai/phobert-base', num_labels=num_labels)


Some weights of RobertaForTokenClassification were not initialized from the model checkpoint at vinai/phobert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [36]:
!pip install seqeval -qq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [37]:
import os
from transformers import TrainingArguments, Trainer, AutoModelForTokenClassification, AutoTokenizer
from seqeval.metrics import f1_score, precision_score, recall_score
from datasets import load_dataset  # Nếu bạn cần load dataset từ Huggingface

# Tự động đăng nhập wandb
os.environ["WANDB_API_KEY"] = "fb80f73fcb020dd331c4509a5851a96be928f490"  # Thêm mã API của bạn ở đây

# Load tokenizer và model PhoBERT
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base")
model = AutoModelForTokenClassification.from_pretrained("vinai/phobert-base", num_labels=len(label_list))

# Tải dataset của bạn, nếu cần thiết
# dataset = load_dataset('path_to_your_dataset')

def compute_metrics(p):
    preds = p.predictions.argmax(-1)
    labels = p.label_ids
    true_preds, true_labels = [], []

    for pred, label in zip(preds, labels):
        tmp_preds, tmp_labels = [], []
        for p_i, l_i in zip(pred, label):
            if l_i != -100:
                tmp_preds.append(label_list[p_i])
                tmp_labels.append(label_list[l_i])
        true_preds.append(tmp_preds)
        true_labels.append(tmp_labels)

    return {
        "precision": precision_score(true_labels, true_preds),
        "recall": recall_score(true_labels, true_preds),
        "f1": f1_score(true_labels, true_preds),
    }

training_args = TrainingArguments(
    output_dir="./ner_phobert",
    do_train=True,
    do_eval=True,  # Bật việc đánh giá
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    save_strategy="epoch",
    report_to=["wandb"],  # Tích hợp với WandB
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,  # Dataset huấn luyện
    eval_dataset=dev_dataset,  # Dataset đánh giá
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

# Huấn luyện và đánh giá
trainer.train()  # Đây sẽ tự động thực hiện việc huấn luyện qua các epoch và tính toán metrics
eval_metrics = trainer.evaluate()  # Đánh giá mô hình sau huấn luyện
print(eval_metrics)


Some weights of RobertaForTokenClassification were not initialized from the model checkpoint at vinai/phobert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-37-2a570de48a72>:50: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: vkhlinh (vkhlinh-l) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
10,1.581300
20,0.911000
30,0.665900
40,0.504000
50,0.426400
60,0.401600
70,0.320700
80,0.201900
90,0.205600
100,0.166600


{'eval_loss': 0.06501360982656479, 'eval_precision': 0.9435377730981838, 'eval_recall': 0.9603482920294709, 'eval_f1': 0.9518688176326097, 'eval_runtime': 13.0547, 'eval_samples_per_second': 153.202, 'eval_steps_per_second': 9.575, 'epoch': 3.0}


In [38]:
trainer.save_model("./phobert-ner-finetuned")
tokenizer.save_pretrained("./phobert-ner-finetuned")


('./phobert-ner-finetuned/tokenizer_config.json',
 './phobert-ner-finetuned/special_tokens_map.json',
 './phobert-ner-finetuned/vocab.txt',
 './phobert-ner-finetuned/bpe.codes',
 './phobert-ner-finetuned/added_tokens.json')

In [39]:
# Sau khi huấn luyện xong, đánh giá mô hình trên test set

# Tạo dataset từ file test (dựa vào NERDataset bạn đã định nghĩa từ trước)
# Assuming label_map is the correct mapping, otherwise replace with the correct variable
label_to_id = label_map
test_dataset = NERDataset(test_features) # Modify this line to use test_features

# Đánh giá mô hình trên test set
test_metrics = trainer.evaluate(test_dataset)

# In kết quả
print("\n====== Đánh giá trên TEST set ======")
print(f"Loss: {test_metrics['eval_loss']:.4f}")
print(f"Precision: {test_metrics['eval_precision']:.4f}")
print(f"Recall: {test_metrics['eval_recall']:.4f}")
print(f"F1 Score: {test_metrics['eval_f1']:.4f}")


====== Đánh giá trên TEST set ======
Loss: 0.0755
Precision: 0.9326
Recall: 0.9501
F1 Score: 0.9413


In [41]:
from seqeval.metrics import classification_report

# Lấy nhãn thực và nhãn dự đoán từ tập test
predictions, labels, _ = trainer.predict(test_dataset)
preds = predictions.argmax(-1)  # Chọn nhãn có xác suất cao nhất từ predictions

true_labels = []
true_preds = []

# Duyệt qua từng cặp dự đoán và nhãn thực tế
for pred, label in zip(preds, labels):
    true_label = []
    true_pred = []
    # Duyệt qua từng cặp nhãn trong token
    for p_i, l_i in zip(pred, label):
        if l_i != -100:  # Lọc các nhãn không hợp lệ (-100 là nhãn bỏ qua)
            true_label.append(label_list[l_i])  # Lấy nhãn thực tế
            true_pred.append(label_list[p_i])   # Lấy nhãn dự đoán
    true_labels.append(true_label)
    true_preds.append(true_pred)

# In báo cáo chi tiết
print("\n====== Báo cáo chi tiết theo từng thực thể ======")
print(classification_report(true_labels, true_preds, digits=4))



====== Báo cáo chi tiết theo từng thực thể ======
                     precision    recall  f1-score   support

                AGE     0.9688    0.9637    0.9662       579
               DATE     0.9785    0.9915    0.9850      1652
             GENDER     0.9555    0.9804    0.9678       460
                JOB     0.5640    0.6879    0.6198       173
           LOCATION     0.9380    0.9511    0.9445      4435
               NAME     0.8862    0.9308    0.9080       318
       ORGANIZATION     0.8595    0.8962    0.8775       771
         PATIENT_ID     0.9728    0.9855    0.9791      1995
SYMPTOM_AND_DISEASE     0.8708    0.8838    0.8772      1136
     TRANSPORTATION     0.9842    0.9689    0.9765       193

          micro avg     0.9326    0.9501    0.9413     11712
          macro avg     0.8978    0.9240    0.9102     11712
       weighted avg     0.9340    0.9501    0.9419     11712



In [45]:
from transformers import AutoModelForTokenClassification, AutoTokenizer
import torch

# Tải mô hình và tokenizer đã huấn luyện
model = AutoModelForTokenClassification.from_pretrained('./phobert-ner-finetuned')
tokenizer = AutoTokenizer.from_pretrained('./phobert-ner-finetuned')

# Define the label list to map the labels
label_list = ['B-ORGANIZATION', 'B-TRANSPORTATION', 'B-JOB', 'I-PATIENT_ID', 'B-NAME', 'I-DATE', 'O',
              'B-PATIENT_ID', 'I-AGE', 'I-JOB', 'B-DATE', 'I-TRANSPORTATION', 'B-SYMPTOM_AND_DISEASE',
              'I-SYMPTOM_AND_DISEASE', 'B-GENDER', 'I-NAME', 'B-LOCATION', 'I-LOCATION', 'I-ORGANIZATION',
              'B-AGE']

# Create a mapping from integer labels to label names
id2label = {i: label for i, label in enumerate(label_list)}

# Hàm nhận diện thực thể cho một câu nhập vào
def predict_entities(text):
    # Tiền xử lý câu đầu vào
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=128)
    with torch.no_grad():
        # Dự đoán các nhãn
        outputs = model(**inputs)
        logits = outputs.logits

    # Lấy chỉ số nhãn có xác suất cao nhất cho mỗi token
    predictions = torch.argmax(logits, dim=-1)

    # Lấy nhãn dự đoán từ các chỉ số
    predicted_labels = [id2label[label.item()] for label in predictions[0]]

    # Tách từ và nhãn dự đoán
    tokens = tokenizer.convert_ids_to_tokens(inputs['input_ids'][0])

    # In kết quả dễ nhìn hơn
    print("\nTokens and Predicted Labels:")
    print(f"{'Token':<15} {'Predicted Label'}")
    print("-" * 40)
    for token, label in zip(tokens, predicted_labels):
        print(f"{token:<15} {label}")

# Nhập câu vào để thử nghiệm
text = input("Nhập câu để nhận dạng thực thể: ")
predict_entities(text)



Nhập câu để nhận dạng thực thể: Vào ngày 1 tháng 6 năm 2021, bệnh nhân 456 đã được chuyển đến Bệnh viện Chợ Rẫy.

Tokens and Predicted Labels:
Token           Predicted Label
----------------------------------------
<s>             O
Vào             O
ngày            O
1               B-DATE
tháng           I-DATE
6               I-DATE
năm             O
20@@            B-DATE
21@@            O
,               O
bệnh            O
nhân            O
456             B-PATIENT_ID
đã              O
được            O
chuyển          O
đến             O
Bệnh            B-LOCATION
viện            I-LOCATION
Chợ             I-LOCATION
R@@             I-LOCATION
ẫ@@             I-LOCATION
y.              I-LOCATION
</s>            O


In [47]:
!pip install gradio -q


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.9/46.9 MB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 31.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 5.3 MB/s eta 0:00:00


In [48]:
import gradio as gr
from transformers import AutoModelForTokenClassification, AutoTokenizer
import torch

# Tải mô hình và tokenizer đã huấn luyện
model = AutoModelForTokenClassification.from_pretrained('./phobert-ner-finetuned')
tokenizer = AutoTokenizer.from_pretrained('./phobert-ner-finetuned')

# Define the label list to map the labels
label_list = ['B-ORGANIZATION', 'B-TRANSPORTATION', 'B-JOB', 'I-PATIENT_ID', 'B-NAME', 'I-DATE', 'O',
              'B-PATIENT_ID', 'I-AGE', 'I-JOB', 'B-DATE', 'I-TRANSPORTATION', 'B-SYMPTOM_AND_DISEASE',
              'I-SYMPTOM_AND_DISEASE', 'B-GENDER', 'I-NAME', 'B-LOCATION', 'I-LOCATION', 'I-ORGANIZATION',
              'B-AGE']

# Create a mapping from integer labels to label names
id2label = {i: label for i, label in enumerate(label_list)}

# Hàm nhận diện thực thể cho một câu nhập vào
def predict_entities(text):
    # Tiền xử lý câu đầu vào
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=128)
    with torch.no_grad():
        # Dự đoán các nhãn
        outputs = model(**inputs)
        logits = outputs.logits

    # Lấy chỉ số nhãn có xác suất cao nhất cho mỗi token
    predictions = torch.argmax(logits, dim=-1)

    # Lấy nhãn dự đoán từ các chỉ số
    predicted_labels = [id2label[label.item()] for label in predictions[0]]

    # Tách từ và nhãn dự đoán
    tokens = tokenizer.convert_ids_to_tokens(inputs['input_ids'][0])

    # Chuẩn bị kết quả để hiển thị dễ nhìn
    result = []
    for token, label in zip(tokens, predicted_labels):
        result.append(f"{token:<15} {label}")

    return "\n".join(result)


# Tạo giao diện Gradio
iface = gr.Interface(
    fn=predict_entities,
    inputs="text",
    outputs="text",
    title="Entity Recognition Demo",
    description="Nhập vào một câu để nhận diện các thực thể. Ví dụ: 'Bệnh nhân Nguyễn Văn A, 35 tuổi, đến từ Hà Nội.'"
)

# Chạy giao diện
iface.launch()


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://345fda5be59eb5fa1d.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [51]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [53]:
import shutil

# Đường dẫn đến thư mục mô hình đã huấn luyện
model_dir = './phobert-ner-finetuned'

# Đường dẫn lưu mô hình trong Google Drive
drive_model_dir = '/content/drive/MyDrive/phobert-ner-finetuned'

# Sao chép mô hình vào Google Drive
shutil.copytree(model_dir, drive_model_dir)


'/content/drive/MyDrive/phobert-ner-finetuned'